# 0. 設定 Spark

In [1]:
spark

In [2]:
import pandas as pd
import pyspark.pandas as ps
import numpy as np
import gc
import pyspark.sql.functions as sf
from pyspark.context import SparkContext
from pyspark.sql.functions import array_max
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.pandas.config import set_option, reset_option
from dateutil.relativedelta import relativedelta
import surprise

In [3]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
ps.set_option("compute.default_index_type", "distributed")
set_option("compute.ops_on_diff_frames", True)

In [4]:
spark.conf.get("spark.kryoserializer.buffer.max")

'512m'

# 1. 讀取檔案 => tran_ps

In [ ]:
# 讀取檔案
# customers = ps.read_parquet('/user/HM_parquet/customers.parquet')
# articles = ps.read_parquet('/user/HM_parquet/articles.parquet')
tran_ps = ps.read_parquet('/user/HM_parquet/transactions_train.parquet').drop(['price', 'sales_channel_id'], axis=1)

In [ ]:
tran_ps.set_index('t_dat',inplace=True)
tran_ps['start_test'] = ''
tran_ps['split_id'] = ''
tran_ps.head(5)

# 2. 作時間分割 tran_ps => split_data, split_id

In [ ]:
def split(data,train_period=30, test_period=7, stride=30,show_progress=False):
    
    split_data = ps.DataFrame(columns = ['t_dat','customer_id', 'article_id', 'split_id', 'start_test']).set_index('t_dat',inplace=True)

    end_test = data.index.max()
    start_test = end_test - relativedelta(days=test_period)
    start_train = start_test - relativedelta(days=train_period)
    split_id=0

    while start_train >= data.index.min():

        df = data.loc[start_train:end_test]
        df['start_test']=start_test
        df['split_id'] = split_id
        split_data = ps.concat([split_data,df])

        if(show_progress):
            print("Split_id:",split_id,", Train period:",start_train,"-" , start_test, ", test period", start_test, "-", end_test)

        # update dates:
        end_test = end_test - relativedelta(days=stride)
        start_test = end_test - relativedelta(days=test_period)
        start_train = start_test - relativedelta(days=train_period)
        split_id += 1
    
    return split_data, split_id

In [ ]:
split_data, split_id = split(tran_ps,30,7,30,True)

In [ ]:
split_data.reset_index(inplace=True)

In [ ]:
split_data.count()

In [ ]:
split_data.head(5)

# 3. 製作參數表 split_id => para_cross_split

In [ ]:
# 製作參數表 paras_grid
from itertools import product

paras = list(
    product(
        [25,50,100,150,200],
        [20,30,40,50],
        [0.01]
    )
)
paras_grid = ps.DataFrame(paras,columns= ['n_factors','n_epochs','reg_all'])
paras_grid.count()

In [ ]:
# 製作 split_id 表
split_id_ps = ps.DataFrame({'split_id': range(split_id)})
split_id_ps.count()

In [ ]:
# 將 paras_grid 與 split_id 做 cross join
paras_grid['key'] = 1
split_id_ps['key'] = 1

para_cross_split = ps.merge(paras_grid, split_id_ps, on ='key').drop('key')
del split_id_ps
len(para_cross_split)

In [ ]:
# 將 cross join 後的表新增遞增的 group_id 欄位，之後要用來做 pandas_udf 的 groupby
para_cross_split['group_id'] = 0
para_cross_split['group_id'] = np.arange(len(para_cross_split)).tolist()
para_cross_split

In [ ]:
# para_cross_split.to_parquet('/user/HM_parquet/SVD_model/para_cross_split.parquet')

# 4. join 參數表和資料表 split_data, para_cross_split => join_data

In [ ]:
join_data = split_data.join(para_cross_split.set_index('split_id'), on='split_id')
join_data.set_index('t_dat',inplace=True)

In [ ]:
join_data.head(5)

In [ ]:
39919883*20

In [ ]:
join_data.count()

In [ ]:
# join_data.to_parquet('/user/HM_parquet/SVD_model/join_data30.parquet')

In [ ]:
# 刪除用不到的資料表
del split_data, para_cross_split
gc.collect()

# -----------------------------

# 讀取資料表

In [5]:
df = spark.read.option('header','true').parquet('/user/HM_parquet/SVD_model/join_data30.parquet')
# join_data.set_index('t_dat',inplace=True)
df.count()

798397660

In [6]:
df.head(5)

[Row(split_id=19, t_dat=datetime.date(2019, 1, 24), customer_id='000253f6914890557a88d0b91288ce85fae9332dac43ee5445c33e3891df6fd3', article_id=660599007, start_test=datetime.date(2019, 2, 23), n_factors=25, n_epochs=30, reg_all=0.01, group_id=43),
 Row(split_id=19, t_dat=datetime.date(2019, 1, 24), customer_id='000253f6914890557a88d0b91288ce85fae9332dac43ee5445c33e3891df6fd3', article_id=660599007, start_test=datetime.date(2019, 2, 23), n_factors=25, n_epochs=40, reg_all=0.01, group_id=67),
 Row(split_id=19, t_dat=datetime.date(2019, 1, 24), customer_id='000253f6914890557a88d0b91288ce85fae9332dac43ee5445c33e3891df6fd3', article_id=660599007, start_test=datetime.date(2019, 2, 23), n_factors=25, n_epochs=50, reg_all=0.01, group_id=91),
 Row(split_id=19, t_dat=datetime.date(2019, 1, 24), customer_id='000253f6914890557a88d0b91288ce85fae9332dac43ee5445c33e3891df6fd3', article_id=660599007, start_test=datetime.date(2019, 2, 23), n_factors=50, n_epochs=20, reg_all=0.01, group_id=115),
 Row(sp

# surpriseSVD

In [7]:
import pandas as pd
# from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import SVDpp,SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
import numpy as np
import average_precision as metrics
# import ml_metrics as metrics

class surpriseSVD():
    def __init__(self):
        self = self

    def get_top_n(self, predictions, n=12):
        """Return the top-N recommendation for each user from a set of predictions.
        Args:
            predictions(list of Prediction objects): The list of predictions, as
                returned by the test method of an algorithm.
            n(int): The number of recommendation to output for each user. Default
                is 10.
        Returns:
        A dict where keys are user (raw) ids and values are lists of tuples:
            [(raw item id, rating estimation), ...] of size n.
        """

        # First map the predictions to each user.
        top_n = defaultdict(list)
        for uid, iid, true_r, est, _ in predictions:
            top_n[uid].append((iid, est))

        # Then sort the predictions for each user and retrieve the k highest ones.
        for uid, user_ratings in top_n.items():
            user_ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = user_ratings[:n]

        return top_n

    def get_set(self,df):
        reader = Reader(rating_scale=(1, 500))
        data_set = Dataset.load_from_df(df[['customer_id','article_id','rating']], reader)
        return data_set

    def get_rating_set(self,df):
        rating = df[['customer_id','article_id','price']].groupby(['customer_id','article_id']).count().reset_index()
        rating.columns = ['customer_id','article_id','rating']
        rating_set = self.get_set(rating)
        return rating_set


    def train_SVD(self, train_data, test_data, paras={}):

        ## 讀取評分資料為surprise可以訓練的格式
        trainset = self.get_rating_set(train_data)
        testset = self.get_rating_set(test_data)

        ## rmse 需要的資料
        testset2 = [testset.df.loc[i].to_list() for i in range(len(testset.df))]

        ## map@k testing 需要產的資料
        test_data.loc[:,'rating']=0
        test_processed = self.get_set(test_data)
        NA, test2 = train_test_split(test_processed, test_size=1.0)

        # ======= 消費者的實際購買清單 =======
        test_data['article_id'] = test_data['article_id'].astype('str')
        test_uni = test_data.drop_duplicates(subset=['customer_id', 'article_id'], keep='first')
        buy_n = test_uni[['customer_id','article_id']].groupby('customer_id')['article_id'].apply(list).to_dict()

        cust_actual_list = []
        for uid, user_ratings in buy_n.items():
            cust_pred_tuple = (uid, [iid for iid in user_ratings])
            cust_actual_list.append(cust_pred_tuple)

        # ======= 訓練 SVD 模型 =======
        algo = SVD(random_state=42,**paras)

        # 訓練模型
        algo.fit(trainset.build_full_trainset())

        ##### rmse #####
        predictions = algo.test(testset2)
        rmse = accuracy.rmse(predictions)

        ##### map@k #####
        predictions_map = algo.test(test2)
        # est = [i.est for i in predictions_map] 

        ##  消費者的預測清單 
        top_n = self.get_top_n(predictions=predictions_map, n=12)

        cust_pred_list = []
        for uid, user_ratings in top_n.items():
            cust_pred_tuple = (uid, [str(iid) for (iid, _) in user_ratings])
            cust_pred_list.append(cust_pred_tuple)

        final_list = list(zip(cust_actual_list, cust_pred_list))

        # map@k計算 
        mapk_list = []
        for i in range(len(final_list)):
            map_k = metrics.mapk([final_list[i][0][1]],[final_list[i][1][1]],12)
            mapk_list.append(map_k)

        map_k = sum(mapk_list)/len(mapk_list)

        return rmse, map_k

# pandas_udf

In [ ]:
# pdf = df.where("group_id == 43").toPandas()

In [8]:
# pdf['t_dat'] = pd.to_datetime(pdf.t_dat)
# pdf.set_index('t_dat',inplace=True, drop=False)
# pdf.head(5)

In [9]:
def time_split_hyperparameter_search(data):
    paras = {
        'n_factors':data.n_factors.values[0], 
        'n_epochs':data.n_epochs.values[0], 
        'reg_all':data.reg_all.values[0]
    }
    
    data['t_dat'] = pd.to_datetime(data.t_dat)
    data.set_index('t_dat',inplace=True, drop=False)
    dataTrain = data.first('22D')
    dataTest = data.last('6D')
    
    model = surpriseSVD()
    rmse, map12 = model.train_SVD(dataTrain, dataTest, paras)
    
    paras.update({
        'start_test' : data.start_test.values[0],
        'rmse' : rmse,
        'map12' : map12
    })
    
    results = pd.DataFrame([paras])
    
    return results


In [10]:
from pyspark.sql.types import (
    DateType, DoubleType, FloatType, IntegerType, StringType, StructField, StructType
)
# pandas_udf

schema = StructType(
    [
        StructField('start_test', DateType(),True),
        StructField("n_factors", IntegerType(), True),
        StructField("n_epochs", IntegerType(), True),
        StructField('reg_all', FloatType(),True),
        StructField('rmse', FloatType(),True),
        StructField('map12', FloatType(),True),
     ]
)

results = df.groupby('group_id').applyInPandas(time_split_hyperparameter_search, schema)

In [11]:
results.count()

2022-03-30 19:17:04,412 WARN scheduler.TaskSetManager: Lost task 9.0 in stage 7.0 (TID 65) (bdse74.example.com executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1648620987516_0008/container_e26_1648620987516_0008_01_000005/pyspark.zip/pyspark/worker.py", line 603, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1648620987516_0008/container_e26_1648620987516_0008_01_000005/pyspark.zip/pyspark/worker.py", line 423, in read_udfs
    arg_offsets, f = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1648620987516_0008/container_e26_1648620987516_0008_01_000005/pyspark.zip/pyspark/worker.py", line 251, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/hdpf

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1648620987516_0008/container_e26_1648620987516_0008_01_000005/pyspark.zip/pyspark/worker.py", line 603, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1648620987516_0008/container_e26_1648620987516_0008_01_000005/pyspark.zip/pyspark/worker.py", line 423, in read_udfs
    arg_offsets, f = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1648620987516_0008/container_e26_1648620987516_0008_01_000005/pyspark.zip/pyspark/worker.py", line 251, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1648620987516_

2022-03-30 19:17:27,308 WARN scheduler.TaskSetManager: Lost task 1.0 in stage 7.0 (TID 61) (bdse89.example.com executor 6): TaskKilled (Stage cancelled)
2022-03-30 19:17:27,393 WARN scheduler.TaskSetManager: Lost task 14.0 in stage 7.0 (TID 72) (bdse89.example.com executor 6): TaskKilled (Stage cancelled)
2022-03-30 19:17:27,962 WARN scheduler.TaskSetManager: Lost task 39.0 in stage 7.0 (TID 93) (bdse75.example.com executor 8): TaskKilled (Stage cancelled)
2022-03-30 19:17:28,467 WARN scheduler.TaskSetManager: Lost task 6.1 in stage 7.0 (TID 92) (bdse42.example.com executor 7): TaskKilled (Stage cancelled)
2022-03-30 19:17:28,515 WARN scheduler.TaskSetManager: Lost task 2.1 in stage 7.0 (TID 109) (bdse92.example.com executor 9): TaskKilled (Stage cancelled)
2022-03-30 19:17:32,140 WARN scheduler.TaskSetManager: Lost task 8.0 in stage 7.0 (TID 64) (bdse108.example.com executor 10): TaskKilled (Stage cancelled)
2022-03-30 19:17:32,146 WARN scheduler.TaskSetManager: Lost task 16.0 in stag

In [ ]:
results.write.parquet('/user/HM_parquet/SVD_model/para_30.parquet')

# (DataFrame)

## 1. 讀取檔案 /DataFrame

In [ ]:
# # 讀取檔案
# customers = spark.read.option('header','true').parquet('/user/HM_parquet/customers.parquet')
# articles = spark.read.option('header','true').parquet('/user/HM_parquet/articles.parquet')
# transactions = spark.read.option('header','true').parquet('/user/HM_parquet/transactions_train.parquet')

In [ ]:
# transactions.show()

## 2. 將customer_id(字串)轉為customer_index(整數) /DataFrame

In [ ]:
# # 將customers的customer_id轉為數字(buffer要增加到512m)
# toIndex = StringIndexer(inputCol="customer_id", outputCol="customer_index").fit(customers)
# customers = toIndex.transform(customers)
# # customers.head(5)

In [ ]:
# # 將transactions的customer_id轉為數字
# transactions = toIndex.transform(transactions)
# # transactions.head(5)

In [ ]:
# transactions.describe()